In [1]:
import json
import time
import datetime
import threading
import importlib

import shioaji_login
# Need to reload this for some reason that I can't remember.
importlib.reload(shioaji_login)

<module 'shioaji_login' from 'C:\\Users\\pride\\git\\shioaji-coverer\\shioaji_login.py'>

In [2]:
api = shioaji_login.login()

Response Code: 0 | Event Code: 0 | Info: host '203.66.91.161:80', hostname '203.66.91.161:80' IP 203.66.91.161:80 (host 1 of 1) (host connection attempt 1 of 1) (total connection attempt 1 of 1) | Event: Session up
Login with ID N125685958
Activating CA at the path C:\sinopac\Sinopac_ROR.pfx


In [3]:
def stop_price_updater():
    """
    Update stop price every single seconds.
    If the time for now is inside of the range intense time, set the prices to intense version.
    Otherwise, set them to normal version.
    This thread will run as long as the program is running.
    
    :global param profit_stop: (int)
    :global param loss_stop: (int)
    :global param normal_profit_stop: (int)
    :global param normal_loss_stop: (int)
    :global param intense_profit_stop: (int)
    :global param intense_loss_stop: (int)
    
    :return: None
    """
    
    pre_profit_stop = 0
    pre_loss_stop = 0
    
    while(True):
        now = datetime.datetime.now().time()
        
        if(now > intense_begin_time or now < intense_end_time):
            profit_stop = intense_profit_stop
            loss_stop = intense_loss_stop
        else:
            profit_stop = normal_profit_stop
            loss_stop = normal_loss_stop
        
        if(pre_profit_stop != profit_stop or pre_loss_stop != loss_stop):
            
            print(f'Time is now {now.strftime("%H:%M:%S")}')
            print(f'Profit stop has been set to {profit_stop}')
            print(f'Loss stop has been set to {loss_stop}')
        
        pre_profit_stop = profit_stop
        pre_loss_stop = loss_stop
        
        time.sleep(1)
        
    return

In [28]:
def future_code(future_name):
    """
    Return future code based on future name
    
    :param future_name: (str)
    
    :return: future_code (str)
    """
    
    now = datetime.datetime.now().replace(month=12)
    month = now.month
    year = now.year
    first_weekday = now.replace(day=1).weekday()
    
    if(first_weekday < 3):
        third_wednesday = 17 - first_weekday
    else:
        third_wednesday = 24 - first_weekday
    
    print(f'Third_wednesday = {third_wednesday}')
    
    if(
        now.day == third_wednesday and now.time() > datetime.time(13, 30) or
        now.day > third_wednesday
    ):
        month = month + 1
        if(month == 13):
            month = 1
            year += 1
    
    month_to_code = '0ABCDEFGHIJKL'
    
    future_code = future_name
    future_code += month_to_code[month]
    future_code += str(year%10)
    
    return future_code

In [29]:
future_code('TXF')

Third_wednesday = 21


'TXFL2'

In [4]:
# Parsing config.json

with open('config.json') as f:
    config_data = json.load(f)
    
    intense_begin_time = datetime.datetime.strptime(config_data['intense_begin'], '%H:%M').time()
    intense_end_time = datetime.datetime.strptime(config_data['intense_end'], '%H:%M').time()
    
    normal_profit_stop = config_data['normal_profit_stop']
    normal_loss_stop = config_data['normal_loss_stop']
    intense_profit_stop = config_data['intense_profit_stop']
    intense_loss_stop = config_data['intense_loss_stop']
    
    future_name = config_data['future_name']

# profit_stop and loss_stop need to be initialize as None before calling stop_price_updater
profit_stop = None
loss_stop = None

stop_price_updater_thread = threading.Thread(target = stop_price_updater)
stop_price_updater_thread.start()

Time is now 22:50:06
Profit stop has been set to 4
Loss stop has been set to 5
